# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4902, uptime 0:00:21
mini_beamline                    RUNNING   pid 4903, uptime 0:00:21
random_walk                      RUNNING   pid 4904, uptime 0:00:21
random_walk_horiz                RUNNING   pid 4905, uptime 0:00:21
random_walk_vert                 RUNNING   pid 4906, uptime 0:00:21
simple                           RUNNING   pid 4907, uptime 0:00:21
thermo_sim                       RUNNING   pid 4908, uptime 0:00:21
trigger_with_pc                  RUNNING   pid 4909, uptime 0:00:21


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-18 16:21:26
Persistent Unique Scan ID: '457980a6-cc62-4713-952f-be743cea6600'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 16:21:27.0 |     -1.000 |      96236 |
|         2 | 16:21:27.0 |     -0.500 |      97039 |
|         3 | 16:21:27.0 |      0.000 |      97868 |
|         4 | 16:21:27.1 |      0.500 |      96649 |
|         5 | 16:21:27.1 |      1.000 |      94844 |


+-----------+------------+------------+------------+
generator scan ['457980a6'] (scan num: 1)





('457980a6-cc62-4713-952f-be743cea6600',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/457980a6-cc62-4713-952f-be743cea6600-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-18 16:21:27
Persistent Unique Scan ID: '62f32f53-14fc-485c-b1ca-d58f8ad362fa'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 16:21:27.6 |          -1 | 1056537596 |


|         2 | 16:21:27.7 |          -0 | 1054409169 |


|         3 | 16:21:27.8 |           0 | 1053307341 |


|         4 | 16:21:27.9 |           0 | 1054419315 |


|         5 | 16:21:28.0 |           1 | 1056228049 |
+-----------+------------+-------------+------------+
generator scan ['62f32f53'] (scan num: 2)





('62f32f53-14fc-485c-b1ca-d58f8ad362fa',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 16:21:28
Persistent Unique Scan ID: '71326ad9-1c4e-4dfe-a7e0-021374468076'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 16:21:28.5 | 1072586048 |


|         2 | 16:21:29.5 | 1127257928 |


|         3 | 16:21:30.5 | 1110435683 |


...

|        28 | 16:21:55.5 | 1056051633 |


|        29 | 16:21:56.5 | 1072619593 |


|        30 | 16:21:57.5 | 1128533713 |


+-----------+------------+------------+
generator count ['71326ad9'] (scan num: 3)





('71326ad9-1c4e-4dfe-a7e0-021374468076',)